In [1]:
!pip install geopy

You should consider upgrading via the 'c:\users\user\anaconda3\python.exe -m pip install --upgrade pip' command.


In [17]:
!pip install --upgrade pip

  Attempting uninstall: pip
    Found existing installation: pip 20.3
    Uninstalling pip-20.3:
      Successfully uninstalled pip-20.3


ERROR: Could not install packages due to an EnvironmentError: [WinError 5] 액세스가 거부되었습니다: 'C:\\Users\\USER\\AppData\\Local\\Temp\\pip-uninstall-52e0bxa5\\pip.exe'
Consider using the `--user` option or check the permissions.



In [2]:
import requests
import urllib.parse as urlparse
from geopy import distance

In [19]:
naver_KEY_ID = 'nWxD0TpBM0o2UmWor2YT'
naver_KEY_secret = 'yNSdVVOHuN'

In [20]:
class Route:
    def __init__(self, headers={"X-NCP-APIGW-API-KEY-ID": naver_KEY_ID,
           "X-NCP-APIGW-API-KEY": naver_KEY_secret}):
        self.headers = headers
    
    # 각 출발지 -> 위경도 변환
    
    def addr_to_xy(self):
        # 주소값 입력
        self.d1_name = input("출발지 1을 입력하세요. : ")
        self.d2_name = input("출발지 2를 입력하세요. : ")
        # URL 설정
        self.d1_url = "https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode?query={}".format(self.d1_name)
        self.d2_url = "https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode?query={}".format(self.d2_name)
        # 리퀘스트
        self.response1 = requests.get(self.d1_url, headers=self.headers)
        self.response2 = requests.get(self.d2_url, headers=self.headers)
        # JSON 파싱하여 위경도 추출
        self.d1_x = self.response1.json()["addresses"][0]["x"] 
        self.d1_y = self.response1.json()["addresses"][0]["y"]
        self.d2_x = self.response2.json()["addresses"][0]["x"]
        self.d2_y = self.response2.json()["addresses"][0]["y"]
        return self.d1_x, self.d1_y, self.d2_x, self.d2_y
    
    # 출발지간 위경도 -> 경로값 변환
    def road_path(self):
        self.d1_xy = str(self.d1_x) + "," + str(self.d1_y)
        self.d2_xy = str(self.d2_x) + "," + str(self.d2_y)
        self.d_option = "traoptimal"
        self.d_url = "https://naveropenapi.apigw.ntruss.com/map-direction/v1/driving?start={}&goal={}&option={}".format(self.d1_xy, self.d2_xy, self.d_option)
        self.response = requests.get(self.d_url, headers=self.headers)
        self.path = self.response.json()["route"]["traoptimal"][0]["path"]
        self.avg_path = self.response.json()["route"]["traoptimal"][0]["summary"]["distance"] / len(self.path)
        print("거리 : {}m, ".format(self.response.json()["route"]["traoptimal"][0]["summary"]["distance"]), len(self.path), ", ", self.avg_path)
        return self.path
    
    # 출발지간 대중교통 경로
    def trans_path(self, path_type=0):
        self.path_type = path_type
        self.key = 'ODSAY KEY'
        self.url = 'https://api.odsay.com/v1/api/searchPubTransPathT'
        self.params = {'apiKey' : self.key,
                  'SX' : self.d1_x,
                  'SY' : self.d1_y,
                  'EX' : self.d2_x,
                  'EY' : self.d2_y,
                  'SearchPathType' : self.path_type,
                  'OPT':0,
                 }
        self.url = self.url + '?' + urlparse.urlencode(self.params)
        self.response = requests.get(self.url)
        # 경로내 정류장 위경도 추출
        self.transit_count = self.response.json()['result']['path'][0]['info']['busTransitCount'] + self.response.json()['result']['path'][0]['info']['subwayTransitCount']
        self.distance = round(self.response.json()['result']['path'][0]['info']['totalDistance'] / 1000,2)
        self.stop_lat_lng = []
        for i in range(1, 2*self.transit_count, 2):
            self.stop_info = self.response.json()['result']['path'][0]['subPath'][i]['passStopList']['stations']
            self.stop_lat_lng += [(float(a['y']),float(a['x'])) for a in self.stop_info]
        # 경로가 1km이하거나, 정류장이 5개 미만이면 모든 위경도 표출하고, 이외에는 중간 6개의 정류장 위경도만 표출
        if self.distance <= 1 or len(self.stop_lat_lng) <= 5:
            self.meet_point = self.stop_lat_lng
        else:
            self.meet_point = self.stop_lat_lng[len(self.stop_lat_lng)//2-3:len(self.stop_lat_lng)//2+3]
        print('거리: {}km'.format(self.distance))
        return self.meet_point, self.stop_lat_lng


In [21]:
rte = Route()


In [22]:
d1_x, d1_y, d2_x, d2_y = rte.addr_to_xy()
d1_x, d1_y, d2_x, d2_y

출발지 1을 입력하세요. : 서울시 강서구 강서로 426
출발지 2를 입력하세요. : 서울시 도봉구 방학2동 344-5


KeyError: 'addresses'

In [ ]:
# 대중교통 이동
meet_points, all_points =rte.trans_path()


In [ ]:
# 경로내 위, 경도 최대 최소값 구하기
lat_list = [point[0] for point in all_points]
lng_list = [point[1] for point in all_points]


In [ ]:
min(lat_list), max(lat_list), min(lng_list), max(lng_list)


In [ ]:
df = pd.read_csv('mango_r.csv')
df.info()


In [ ]:
df['latlng'] = df.apply(lambda x:(x['lat'], x['lng']), axis=1)
df.tail()


In [ ]:
# 경로 square 내 맛집 추출
direction_square = df[(df['lat']<=max(lat_list)) & (df['lat']>=min(lat_list)) & (df['lng']>=min(lng_list)) & (df['lng']<=max(lng_list))]


In [ ]:
# square 내 경로 1km 이내 맛집 추출
matzip=pd.DataFrame()
for lat_lng in direction_square['latlng']:
    for point in meet_points:
        dis = distance.distance(point, lat_lng).km
        if dis <= 1:
            df_1 = direction_square[direction_square['latlng'] == lat_lng]
            matzip = matzip.append(df_1).reset_index(drop=True)


In [ ]:
matzip


In [40]:
df = pd.read_csv('./df.csv')
df = df.drop(columns='Unnamed: 0')

,fname,name,tel,addr,si,gu,dong,bizhour,category,menu,...,microreview,img,lng,lat,n_rating,b_rating,mg_rating,d_rating,mn_rating,rating
0,박순례 손말이고기 산정집 광화문점,박순례,02-310-9636,서울특별시 종로구 당주동 3-2 선덕빌딩 2층,서울특별시,종로구,당주동,일요일 휴무 명절연휴 휴무 | 매일 11:30~14:00 점심 | 매일 14:00~...,한식,"한우말이고기 21,000 | 한우내장볶음 21,000 | 한우내장전골 25,000 ...",...,[],https://ldb-phinf.pstatic.net/20200930_73/1601...,126.974200,37.572380,4.41,3.53,4.00,1.94,3.54,4.00
2,할매집,할매집,02-735-2608,서울특별시 종로구 내자동 108,서울특별시,종로구,내자동,매일 12:00~21:30,한식,"족발 30,000 | 감자탕(중) 25,000 | 감자탕(대) 30,000",...,"['콩나물 넣은 감자탕', '구수하고 푸짐한 감자탕을 먹을 수 있는 식신로드 맛집'...",http://blogfiles.naver.net/20150910_18/xxxyeji...,126.971475,37.575660,4.30,3.53,3.50,3.15,3.90,3.92
5,도토리브라더스,도토리브라더스,02-737-2224,서울특별시 종로구 신문로2가 1-248,서울특별시,종로구,신문로2가,매일 11:30~21:00 (21:20 영업종료) | 매일 15:00~17:00 브...,한식,"대창덮밥 13,500 | 아보카도 골든 에그 살몬 14,000 | 골든에그 살몬 1...",...,[],https://ldb-phinf.pstatic.net/20190829_277/156...,126.969258,37.572947,4.31,3.53,3.90,3.30,3.54,4.03
6,대접 광화문본점,대접,02-733-4845,서울특별시 종로구 내수동 72,서울특별시,종로구,내수동,평일 11:30~21:00 15-17시 브레이크타임 | 공휴일 11:30~20:00...,한식,"모둠버섯영양밥+제철 국물 18,000 | 백미+소고기고추장찌개 18,000",...,[],https://ldb-phinf.pstatic.net/20200512_30/1589...,126.972373,37.573766,4.70,3.53,3.68,1.94,3.54,4.07
12,서촌계단집,서촌계단집,02-737-8412,서울특별시 종로구 내자동 11-1,서울특별시,종로구,내자동,평일 15:00~01:00 동절기 | 주말 13:30~01:00 하절기,한식,"벌교왕꼬막 25,000 | 통영생굴회 15,000 | 해물라면 7,000 | 참소라...",...,"['조개찜과 문어 숙회가 맛있는 곳', '신선한 해산물과 깔끔한 잔치국수', '식신...",https://ldb-phinf.pstatic.net/20151104_22/1446...,126.971466,37.576447,4.25,3.53,4.30,3.45,3.54,4.11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10910,영화정해장국,영화정해장국,02-476-1968,서울특별시 강동구 길동 413-20,서울특별시,강동구,길동,매일 05:00~15:00 월요일 휴무,한식,"해장국 7,000",...,"['진짜들만 가는 길동 해장국집', '선지해장국만 파는곳', '선지국 맛있는 곳',...",https://ldb-phinf.pstatic.net/20200319_117/158...,127.138300,37.535817,4.45,3.53,3.68,2.60,3.54,3.99
10914,양촌리,양촌리,02-485-9292,서울특별시 강동구 길동 417,서울특별시,강동구,길동,매일 00:00~24:00,한식,"돼지갈비 17,000 | 김치찌개 8,000 | 고등어조림 10,000 | 묵은지찜...",...,"['천호에서 돼지갈비로 유명한 이곳', '양념갈비 정육식당 늦게까지하는곳', '돼지...",https://ldb-phinf.pstatic.net/20190214_272/155...,127.139180,37.534764,4.32,3.53,3.68,1.94,3.54,3.88
10915,참사랑해,참사랑해,02-473-1337,서울특별시 강동구 길동 366-14,서울특별시,강동구,길동,매일 12:00~00:30,한식,"참치숯불구이 1kg 43,000 | 참치숯불구이 1.5kg 65,000 | 참치숯불...",...,[],https://ldb-phinf.pstatic.net/20190719_215/156...,127.141079,37.539462,4.53,3.53,3.68,0.75,3.54,3.89
10918,도니도리곱창 길동점,도니도리곱창,02-484-9991,서울특별시 강동구 길동 390-23,서울특별시,강동구,길동,매일 17:00~02:00 | 일요일 17:00~12:00 | 월요일 휴무,한식,"막창소금구이 12,000 | 막창양념구이 12,000 | 야채곱창 11,000 | ...",...,[],https://ldb-phinf.pstatic.net/20190401_231/155...,127.140432,37.537137,4.46,3.53,3.68,1.94,3.54,3.95


In [59]:
37.572380 + 1.000000
37.572380 - 1.000000

36.57238

In [79]:
df.lng <= 30.572380

0        False
1        False
2        False
3        False
4        False
         ...  
10917    False
10918    False
10919    False
10920    False
10921    False
Name: lng, Length: 10922, dtype: bool

In [85]:
df[df.lng < 36.972380]

,fname,name,tel,addr,si,gu,dong,bizhour,category,menu,...,img,lng,lat,n_rating,b_rating,mg_rating,d_rating,mn_rating,rating,recommend
9,4.5,4.5,4.5,4.5,4.5,4.5,4.5,4.5,4.5,4.5,...,4.5,4.5,4.5,4.5,4.5,4.5,1.94,4.5,4.35,0
10,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,...,4.0,4.0,4.0,4.0,4.0,4.0,1.94,4.0,3.89,0
22,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,...,4.0,4.0,4.0,4.0,4.0,4.0,1.94,4.0,3.89,0
26,4.5,4.5,4.5,4.5,4.5,4.5,4.5,4.5,4.5,4.5,...,4.5,4.5,4.5,4.5,4.5,4.5,1.94,4.5,4.35,0
38,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,...,4.0,4.0,4.0,4.0,4.0,4.0,1.94,4.0,3.89,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10777,4.5,4.5,4.5,4.5,4.5,4.5,4.5,4.5,4.5,4.5,...,4.5,4.5,4.5,4.5,4.5,4.5,1.94,4.5,4.35,0
10801,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,...,3.0,3.0,3.0,3.0,3.0,3.0,1.94,3.0,2.95,0
10821,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,...,3.0,3.0,3.0,3.0,3.0,3.0,1.94,3.0,2.95,0
10840,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,...,3.0,3.0,3.0,3.0,3.0,3.0,1.94,3.0,2.95,0


In [80]:
df[(df.lng <= 36.572380) | (df.lng >= 36.572380)]

,fname,name,tel,addr,si,gu,dong,bizhour,category,menu,...,img,lng,lat,n_rating,b_rating,mg_rating,d_rating,mn_rating,rating,recommend
0,박순례 손말이고기 산정집 광화문점,박순례,02-310-9636,서울특별시 종로구 당주동 3-2 선덕빌딩 2층,서울특별시,종로구,당주동,일요일 휴무 명절연휴 휴무 | 매일 11:30~14:00 점심 | 매일 14:00~...,한식,"한우말이고기 21,000 | 한우내장볶음 21,000 | 한우내장전골 25,000 ...",...,https://ldb-phinf.pstatic.net/20200930_73/1601...,126.974200,37.572380,4.41,3.53,4.00,1.94,3.54,4.00,High
1,한옥 달,한옥,010-8706-3392,서울특별시 종로구 체부동 68,서울특별시,종로구,체부동,평일 11:30~22:00 (L.O. 20:30) | 평일 15:30~17:00 B...,양식,"단호박 크림 스프 15,000 | 아보카도 허머스 16,000 | 리코타 치즈 샐러...",...,https://ldb-phinf.pstatic.net/20190808_53/1565...,126.971983,37.576946,4.32,3.53,3.68,1.94,3.54,3.88,0
2,할매집,할매집,02-735-2608,서울특별시 종로구 내자동 108,서울특별시,종로구,내자동,매일 12:00~21:30,한식,"족발 30,000 | 감자탕(중) 25,000 | 감자탕(대) 30,000",...,http://blogfiles.naver.net/20150910_18/xxxyeji...,126.971475,37.575660,4.30,3.53,3.50,3.15,3.90,3.92,High
3,디퍼카페테리아,디퍼카페테리아,010-9160-4482,서울특별시 종로구 체부동 2-2,서울특별시,종로구,체부동,주말 12:00~19:30 | 수요일 12:00~19:30 | 목요일 12:00~1...,디저트,"아메리카노 3,500 | 카페라떼 4,000 | 그라니따 5,500 | 스폴리아텔라...",...,https://ldb-phinf.pstatic.net/20200908_251/159...,126.970393,37.579038,4.48,3.53,3.68,1.94,3.54,3.96,0
4,후라토식당 경복궁 본점,후라토식당,02-730-7090,서울특별시 종로구 당주동 5,서울특별시,종로구,당주동,매일 11:30~22:00 평일 브레이크타임 15:00-17:00,일식,"규카츠 15,000 | 반숙 오므라이스 11,000 | 야마가타 민치카레 12,00...",...,https://ldb-phinf.pstatic.net/20200804_260/159...,126.974311,37.572032,4.67,3.53,4.10,1.94,3.54,4.16,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10917,반피셋,반피셋,010-5448-9476,서울특별시 강동구 길동 366-1,서울특별시,강동구,길동,"매일 11:30~21:30 Break Time 15:00~17:00 매주 일요일,월...",동남아식,"꾸웨이띠여우 느아 뚠 9,000 | 똠얌 꾸웨이띠여우 10,000 | 꾸웨이띠여우 ...",...,https://ldb-phinf.pstatic.net/20191109_241/157...,127.141008,37.539458,4.61,3.53,3.68,1.10,3.54,3.96,0
10918,도니도리곱창 길동점,도니도리곱창,02-484-9991,서울특별시 강동구 길동 390-23,서울특별시,강동구,길동,매일 17:00~02:00 | 일요일 17:00~12:00 | 월요일 휴무,한식,"막창소금구이 12,000 | 막창양념구이 12,000 | 야채곱창 11,000 | ...",...,https://ldb-phinf.pstatic.net/20190401_231/155...,127.140432,37.537137,4.46,3.53,3.68,1.94,3.54,3.95,High
10919,동남집,동남집,02-477-0590,서울특별시 강동구 길동 458-3,서울특별시,강동구,길동,매일 00:00~24:00 월요일 오전 10시 오픈/일요일 오후10시 마감,한식,"양지곰탕 8,000 | 특곰탕 10,000 | 도가니탕 13,000 | 옛날불고기(...",...,https://ldb-phinf.pstatic.net/20191225_218/157...,127.138280,37.535022,4.27,3.53,3.68,1.94,3.54,3.85,High
10920,라무진 길동점,라무진,02-6397-7299,서울특별시 강동구 길동 414-10,서울특별시,강동구,길동,"매일 17:00~24:00 일요일 정기휴무, 라스트오더 마감1시간전",중식,"양갈비 27,000",...,http://ldb.phinf.naver.net/20190412_142/155506...,127.137484,37.536099,4.40,3.53,3.68,1.50,3.54,3.89,0


In [41]:
def search:
    df[df['lng'] < 37.572380 & df['lng'] > 37.572380 + ]
    df[df['lat'] == 126.974200]
    df[df['category'] == '한식']
    df[df['category'] == '한식']

,fname,name,tel,addr,si,gu,dong,bizhour,category,menu,...,microreview,img,lng,lat,n_rating,b_rating,mg_rating,d_rating,mn_rating,rating
0,박순례 손말이고기 산정집 광화문점,박순례,02-310-9636,서울특별시 종로구 당주동 3-2 선덕빌딩 2층,서울특별시,종로구,당주동,일요일 휴무 명절연휴 휴무 | 매일 11:30~14:00 점심 | 매일 14:00~...,한식,"한우말이고기 21,000 | 한우내장볶음 21,000 | 한우내장전골 25,000 ...",...,[],https://ldb-phinf.pstatic.net/20200930_73/1601...,126.974200,37.572380,4.41,3.53,4.00,1.94,3.54,4.00
1,한옥 달,한옥,010-8706-3392,서울특별시 종로구 체부동 68,서울특별시,종로구,체부동,평일 11:30~22:00 (L.O. 20:30) | 평일 15:30~17:00 B...,양식,"단호박 크림 스프 15,000 | 아보카도 허머스 16,000 | 리코타 치즈 샐러...",...,[],https://ldb-phinf.pstatic.net/20190808_53/1565...,126.971983,37.576946,4.32,3.53,3.68,1.94,3.54,3.88
2,할매집,할매집,02-735-2608,서울특별시 종로구 내자동 108,서울특별시,종로구,내자동,매일 12:00~21:30,한식,"족발 30,000 | 감자탕(중) 25,000 | 감자탕(대) 30,000",...,"['콩나물 넣은 감자탕', '구수하고 푸짐한 감자탕을 먹을 수 있는 식신로드 맛집'...",http://blogfiles.naver.net/20150910_18/xxxyeji...,126.971475,37.575660,4.30,3.53,3.50,3.15,3.90,3.92
3,디퍼카페테리아,디퍼카페테리아,010-9160-4482,서울특별시 종로구 체부동 2-2,서울특별시,종로구,체부동,주말 12:00~19:30 | 수요일 12:00~19:30 | 목요일 12:00~1...,디저트,"아메리카노 3,500 | 카페라떼 4,000 | 그라니따 5,500 | 스폴리아텔라...",...,[],https://ldb-phinf.pstatic.net/20200908_251/159...,126.970393,37.579038,4.48,3.53,3.68,1.94,3.54,3.96
4,후라토식당 경복궁 본점,후라토식당,02-730-7090,서울특별시 종로구 당주동 5,서울특별시,종로구,당주동,매일 11:30~22:00 평일 브레이크타임 15:00-17:00,일식,"규카츠 15,000 | 반숙 오므라이스 11,000 | 야마가타 민치카레 12,00...",...,[],https://ldb-phinf.pstatic.net/20200804_260/159...,126.974311,37.572032,4.67,3.53,4.10,1.94,3.54,4.16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10917,반피셋,반피셋,010-5448-9476,서울특별시 강동구 길동 366-1,서울특별시,강동구,길동,"매일 11:30~21:30 Break Time 15:00~17:00 매주 일요일,월...",동남아식,"꾸웨이띠여우 느아 뚠 9,000 | 똠얌 꾸웨이띠여우 10,000 | 꾸웨이띠여우 ...",...,[],https://ldb-phinf.pstatic.net/20191109_241/157...,127.141008,37.539458,4.61,3.53,3.68,1.10,3.54,3.96
10918,도니도리곱창 길동점,도니도리곱창,02-484-9991,서울특별시 강동구 길동 390-23,서울특별시,강동구,길동,매일 17:00~02:00 | 일요일 17:00~12:00 | 월요일 휴무,한식,"막창소금구이 12,000 | 막창양념구이 12,000 | 야채곱창 11,000 | ...",...,[],https://ldb-phinf.pstatic.net/20190401_231/155...,127.140432,37.537137,4.46,3.53,3.68,1.94,3.54,3.95
10919,동남집,동남집,02-477-0590,서울특별시 강동구 길동 458-3,서울특별시,강동구,길동,매일 00:00~24:00 월요일 오전 10시 오픈/일요일 오후10시 마감,한식,"양지곰탕 8,000 | 특곰탕 10,000 | 도가니탕 13,000 | 옛날불고기(...",...,[],https://ldb-phinf.pstatic.net/20191225_218/157...,127.138280,37.535022,4.27,3.53,3.68,1.94,3.54,3.85
10920,라무진 길동점,라무진,02-6397-7299,서울특별시 강동구 길동 414-10,서울특별시,강동구,길동,"매일 17:00~24:00 일요일 정기휴무, 라스트오더 마감1시간전",중식,"양갈비 27,000",...,[],http://ldb.phinf.naver.net/20190412_142/155506...,127.137484,37.536099,4.40,3.53,3.68,1.50,3.54,3.89


In [70]:
condition = [
    (df['category'] == '한식'),
    (df['lat'] == 126.974200),
    (df['lat'] == 37.572380),
]


0        High
1           0
2        High
3           0
4           0
         ... 
10917       0
10918    High
10919    High
10920       0
10921       0
Name: recommend, Length: 10922, dtype: object

In [ ]:
conditionlist = [
    (df['Salary'] >= 500) ,
    (df['Salary'] >= 300) & (df['Salary'] <300),
    (df['Salary'] <= 300)]
choicelist = ['High', 'Mid', 'Low']
df['Salary_Range'] = np.select(conditionlist, choicelist, default='Not Specified')


In [100]:
cd0 = df.category == '한식'
cd1 = df.lat < 37.572380
cd2 = df.lat > 37.562380
cd3 = df.lng > 126.974200
cd4 = df.lng > 126.964200
cd_all = cd0 & cd1 & cd2 & cd3 & cd4 & cd5
df[cd_all].sort_values(by='rating', ascending=True).head()

,fname,name,tel,addr,si,gu,dong,bizhour,category,menu,...,img,lng,lat,n_rating,b_rating,mg_rating,d_rating,mn_rating,rating,recommend
2409,시간을 파는 고깃집 스무날,시간을,02-2291-5050,서울특별시 성동구 홍익동 398,서울특별시,성동구,홍익동,월요일 16:00~00:00 | 화요일 16:00~00:00 | 수요일 16:00~...,한식,"스무날 삼겹 15,000 | 스무날 목살 15,000 | 스무날 어깨살 13,000...",...,https://ldb-phinf.pstatic.net/20201006_101/160...,127.030349,37.565694,4.58,3.53,3.68,1.94,3.54,4.01,High
1113,장흥집,장흥집,02-6959-3521,서울특별시 중구 황학동 2153,서울특별시,중구,황학동,금요일 14:30~00:00 | 목요일 14:30~00:00 | 수요일 14:30~...,한식,"냉동삼겹살(1인분200g) 13,000 | 생삼겹살 180g 14,000 | 냉동목...",...,https://ldb-phinf.pstatic.net/20200406_102/158...,127.023667,37.570437,4.58,3.53,3.68,1.94,3.54,4.01,High
1053,은마난로,은마난로,02-2231-3450,서울특별시 중구 신당동 235-2 1층,서울특별시,중구,신당동,금요일 14:00~21:00 코로나 2단계 적용기간 동안 | 목요일 14:00~21...,한식,"소 모둠구이(마늘) 19,000 | 소 모둠구이(양념) 19,000 | 돼지 모둠구...",...,https://ldb-phinf.pstatic.net/20200615_241/159...,127.013418,37.564579,4.58,3.53,3.68,1.94,3.54,4.01,High
3596,한양닭도리탕,한양닭도리탕,02-2242-4805,서울특별시 동대문구 장안동 344-3,서울특별시,동대문구,장안동,매일 17:00~07:00,한식,"닭도리탕 30,000 | 곱창전골 30,000 | 묵은지닭도리탕 37,000 | 닭...",...,http://blogfiles.naver.net/20160330_31/hahobba...,127.071909,37.570736,4.61,3.53,3.68,1.94,3.54,4.02,High
3579,무쏘 답십리점,무쏘,02-2216-1400,서울특별시 동대문구 답십리동 266-1 2층 201호,서울특별시,동대문구,답십리동,매일 11:00~22:00,한식,"무한리필소고기 B set 22,800 | 무한리필소고기 C set 23,800 | ...",...,https://ldb-phinf.pstatic.net/20200722_47/1595...,127.057296,37.568281,4.60,3.53,3.68,1.94,3.54,4.02,High
